In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

class Analisis5CAutomatico:
    def __init__(self, ruta_archivo):
        self.ruta_archivo = ruta_archivo
        self.df = None
        self.C5 = {}
        self.pesos_automaticos = {}
        
    def cargar_y_preparar_datos(self):
        """Cargar datos y crear variable target"""
        print("📊 Cargando datos...")
        self.df = pd.read_csv(self.ruta_archivo)
        
        # Crear variable target
        self.df["target"] = self.df["diasmora"].apply(lambda x: 1 if x > 7 else 0)
        
        # Eliminar columnas que no se usarán
        columnas_eliminar = ["nosocio", "nocredito", "sucursal", "diasmora"]
        self.df = self.df.drop(columns=[col for col in columnas_eliminar 
                                      if col in self.df.columns], errors='ignore')
        
        print(f"✅ Datos cargados: {self.df.shape[0]} registros, {self.df.shape[1]} variables")
        return self
    
    def definir_categorias_5c_version(self):
        """Definir las categorías de las 5C SIN pesos predefinidos"""
        self.C5 = {
            "Carácter": {
                "variables": ["reputación_localidad", "exp_cred_externa", "exp_cred_interna", "renovadoestructurado"],
                "interpretacion": "Historial crediticio y comportamiento de pago"
            },
            "Capacidad": {
                "variables": ["ingresos_ordinarios", "num_dependientes", "cap_pago", "empleo"],
                "interpretacion": "Capacidad de pago y estabilidad financiera"
            },
            "Capital": {
                "variables": ["monto", "tasaordinaria", "tasamoratoria", "deudas"],
                "interpretacion": "Patrimonio y recursos propios"
            },
            "Condiciones": {
                "variables": ["producto", "clasificacion", "comercio_en_region", "permiso_para_funcionar", "plazocredito", "frecuenciacapint"],
                "interpretacion": "Condiciones del préstamo y entorno"
            },
            "Colateral": {
                "variables": ["montogarantialiq", "montogarantiapre", "montogarantiahipo"],
                "interpretacion": "Garantías y respaldos"
            }
        }
        
        # Filtrar solo variables existentes
        for categoria, info in self.C5.items():
            variables_existentes = [v for v in info["variables"] if v in self.df.columns]
            self.C5[categoria]["variables"] = variables_existentes
            print(f"   {categoria}: {len(variables_existentes)} variables")
        
        return self
    
    def definir_categorias_5c(self):
        """Definir las categorías de las 5C SIN pesos predefinidos"""
        self.C5 = {
            "Carácter": {
                "variables": ["reputación_localidad", "antigudad_domicilio", "antiguedad_actividad", "exp_cred_externa", "exp_cred_interna"],
                "interpretacion": "Historial crediticio y comportamiento de pago"
            },
            "Capacidad": {
                "variables": ["cap_pago"],
                "interpretacion": "Capacidad de pago y estabilidad financiera"
            },
            "Capital": {
                "variables": ["tipo_casa", "deudas"],
                "interpretacion": "Patrimonio y recursos propios"
            },
            "Condiciones": {
                "variables": ["permiso_para_funcionar", "comercio_en_region", "empleo"],
                "interpretacion": "Condiciones del préstamo y entorno"
            },
            "Colateral": {
                "variables": ["montogarantialiq", "montogarantiapre", "montogarantiahipo"],
                "interpretacion": "Garantías y respaldos"
            }
        }
        
        # Filtrar solo variables existentes
        for categoria, info in self.C5.items():
            variables_existentes = [v for v in info["variables"] if v in self.df.columns]
            self.C5[categoria]["variables"] = variables_existentes
            print(f"   {categoria}: {len(variables_existentes)} variables")
        
        return self
    
    def preprocesar_variables(self):
        """Preprocesar variables para análisis"""
        df_encoded = self.df.copy()
        
        # Codificar variables categóricas
        label_encoders = {}
        for col in df_encoded.select_dtypes(include=['object']).columns:
            if col != 'target':
                le = LabelEncoder()
                df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
                label_encoders[col] = le
        
        # Escalar variables numéricas (excepto target)
        numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns.drop('target', errors='ignore')
        if len(numeric_cols) > 0:
            scaler = StandardScaler()
            df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])
        
        return df_encoded, label_encoders
    
    def calcular_pesos_automaticos(self, mi_df):
        """Calcular pesos automáticos para cada categoría basado en los datos"""
        print("⚖️ Calculando pesos automáticos para las 5C...")
        
        # Método 1: Basado en Mutual Information promedio por categoría
        pesos_mi = {}
        total_mi = 0
        
        for categoria, info in self.C5.items():
            variables_categoria = info["variables"]
            mi_categoria = mi_df[mi_df['Variable'].isin(variables_categoria)]
            
            if len(mi_categoria) > 0:
                mi_promedio = mi_categoria['MI_Score'].mean()
                pesos_mi[categoria] = mi_promedio
                total_mi += mi_promedio
            else:
                pesos_mi[categoria] = 0
        
        # Normalizar pesos
        if total_mi > 0:
            pesos_mi = {k: v/total_mi for k, v in pesos_mi.items()}
        
        # Método 2: Basado en Random Forest (importancia de características)
        print("🌲 Calculando importancia con Random Forest...")
        df_encoded, _ = self.preprocesar_variables()
        X = df_encoded.drop(columns=['target'], errors='ignore')
        y = df_encoded['target']
        
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X, y)
        
        feature_importance = pd.DataFrame({
            'Variable': X.columns,
            'RF_Importance': rf.feature_importances_
        })
        
        pesos_rf = {}
        total_rf = 0
        
        for categoria, info in self.C5.items():
            variables_categoria = info["variables"]
            rf_categoria = feature_importance[feature_importance['Variable'].isin(variables_categoria)]
            
            if len(rf_categoria) > 0:
                rf_promedio = rf_categoria['RF_Importance'].mean()
                pesos_rf[categoria] = rf_promedio
                total_rf += rf_promedio
            else:
                pesos_rf[categoria] = 0
        
        # Normalizar pesos RF
        if total_rf > 0:
            pesos_rf = {k: v/total_rf for k, v in pesos_rf.items()}
        
        # Método 3: Basado en correlación con el target
        print("📊 Calculando correlaciones...")
        correlaciones = {}
        df_numeric = self.df.select_dtypes(include=[np.number])
        
        if 'target' in df_numeric.columns:
            for categoria, info in self.C5.items():
                variables_categoria = [v for v in info["variables"] if v in df_numeric.columns]
                if variables_categoria:
                    corr_promedio = df_numeric[variables_categoria].corrwith(df_numeric['target']).abs().mean()
                    correlaciones[categoria] = corr_promedio if not np.isnan(corr_promedio) else 0
                else:
                    correlaciones[categoria] = 0
            
            total_corr = sum(correlaciones.values())
            if total_corr > 0:
                correlaciones = {k: v/total_corr for k, v in correlaciones.items()}
        
        # Combinar métodos (promedio ponderado de los tres métodos)
        pesos_combinados = {}
        for categoria in self.C5.keys():
            peso_final = 0
            count = 0
            
            if categoria in pesos_mi and pesos_mi[categoria] > 0:
                peso_final += pesos_mi[categoria]
                count += 1
            
            if categoria in pesos_rf and pesos_rf[categoria] > 0:
                peso_final += pesos_rf[categoria]
                count += 1
            
            if categoria in correlaciones and correlaciones[categoria] > 0:
                peso_final += correlaciones[categoria]
                count += 1
            
            if count > 0:
                pesos_combinados[categoria] = peso_final / count
            else:
                pesos_combinados[categoria] = 1.0 / len(self.C5)  # Distribución equitativa
        
        # Normalizar pesos finales
        total_combinado = sum(pesos_combinados.values())
        if total_combinado > 0:
            pesos_combinados = {k: v/total_combinado for k, v in pesos_combinados.items()}
        
        self.pesos_automaticos = pesos_combinados
        
        # Mostrar pesos calculados
        print("\n📈 PESOS AUTOMÁTICOS CALCULADOS:")
        for categoria, peso in self.pesos_automaticos.items():
            print(f"   {categoria}: {peso:.3f} ({peso*100:.1f}%)")
        
        return pesos_combinados, pesos_mi, pesos_rf, correlaciones
    
    def calcular_importancia_variables(self):
        """Calcular importancia de variables usando Mutual Information"""
        print("🔍 Calculando importancia de variables...")
        
        df_encoded, _ = self.preprocesar_variables()
        X = df_encoded.drop(columns=['target'], errors='ignore')
        y = df_encoded['target']
        
        # Calcular Mutual Information
        mi_scores = mutual_info_classif(X, y, random_state=42)
        mi_df = pd.DataFrame({
            'Variable': X.columns,
            'MI_Score': mi_scores
        }).sort_values('MI_Score', ascending=False)
        
        return mi_df
    
    def calcular_score_por_categoria(self, mi_df):
        """Calcular score para cada categoría de las 5C usando pesos automáticos"""
        print("📈 Calculando scores por categoría 5C...")
        
        resultados = {}
        
        for categoria, info in self.C5.items():
            variables_categoria = info["variables"]
            peso_categoria = self.pesos_automaticos.get(categoria, 0.2)  # Default 20% si no existe
            
            # Filtrar importancia para variables de esta categoría
            mi_categoria = mi_df[mi_df['Variable'].isin(variables_categoria)]
            
            if len(mi_categoria) > 0:
                # Score basado en importancia promedio normalizada
                importancia_promedio = mi_categoria['MI_Score'].mean()
                importancia_maxima = mi_df['MI_Score'].max()
                score_normalizado = (importancia_promedio / importancia_maxima) * 100 if importancia_maxima > 0 else 0
                
                # Aplicar peso automático de la categoría
                score_ponderado = score_normalizado * peso_categoria
                
                # Calcular contribución individual de variables
                contribuciones = {}
                for _, row in mi_categoria.iterrows():
                    var = row['Variable']
                    contrib = (row['MI_Score'] / importancia_maxima) * 100 if importancia_maxima > 0 else 0
                    contribuciones[var] = contrib
                
                resultados[categoria] = {
                    'score_raw': score_normalizado,
                    'score_ponderado': score_ponderado,
                    'peso_automatico': peso_categoria,
                    'num_variables': len(mi_categoria),
                    'variables_analizadas': variables_categoria,
                    'importancia_promedio': importancia_promedio,
                    'contribuciones_variables': contribuciones,
                    'interpretacion': info['interpretacion']
                }
            else:
                resultados[categoria] = {
                    'score_raw': 0,
                    'score_ponderado': 0,
                    'peso_automatico': peso_categoria,
                    'num_variables': 0,
                    'variables_analizadas': [],
                    'importancia_promedio': 0,
                    'contribuciones_variables': {},
                    'interpretacion': info['interpretacion']
                }
        
        return resultados
    
    def calcular_riesgo_por_valor(self):
        """Calcular riesgo por valor de variables categóricas"""
        print("🎯 Calculando riesgo por valor...")
        
        pesos_valores = []
        for col in self.df.select_dtypes(include=['object']).columns:
            if col != 'target':
                tabla_riesgo = self.df.groupby(col).agg({
                    'target': ['mean', 'count']
                }).reset_index()
                
                tabla_riesgo.columns = ['Valor', 'Tasa_morosos', 'Conteo']
                tabla_riesgo['Variable'] = col
                tabla_riesgo['Riesgo_relativo'] = (tabla_riesgo['Tasa_morosos'] / 
                                                 self.df['target'].mean()) * 100
                
                pesos_valores.append(tabla_riesgo)
        
        return pd.concat(pesos_valores, ignore_index=True) if pesos_valores else pd.DataFrame()
    
    def generar_reporte_5c(self, resultados_5c, pesos_detalle):
        """Generar reporte completo de las 5C"""
        print("\n" + "="*70)
        print("📊 REPORTE COMPLETO 5C DEL CRÉDITO (PESOS AUTOMÁTICOS)")
        print("="*70)
        
        total_score = sum([info['score_ponderado'] for info in resultados_5c.values()])
        
        # Mostrar metodología de pesos
        print("\n🔧 METODOLOGÍA DE PESOS:")
        print("   - Mutual Information: Basado en dependencia estadística")
        print("   - Random Forest: Basado en importancia en clasificación")
        print("   - Correlación: Basado en relación lineal con morosidad")
        
        for categoria, info in resultados_5c.items():
            print(f"\n🔹 {categoria.upper()}")
            print(f"   Score: {info['score_raw']:.2f} / 100")
            print(f"   Score ponderado: {info['score_ponderado']:.2f}")
            print(f"   Peso automático: {info['peso_automatico']*100:.1f}%")
            print(f"   Variables analizadas: {info['num_variables']}")
            print(f"   Interpretación: {info['interpretacion']}")
            
            # Mostrar contribución de variables individuales
            if info['contribuciones_variables']:
                print("   Contribución por variable:")
                for var, contrib in info['contribuciones_variables'].items():
                    print(f"     - {var}: {contrib:.2f}")
            
            # Recomendación basada en el score
            if info['score_raw'] >= 70:
                estado = "✅ FUERTE"
            elif info['score_raw'] >= 40:
                estado = "⚠️  MODERADO"
            else:
                estado = "❌ DÉBIL"
            print(f"   Estado: {estado}")
        
        print(f"\n🎯 SCORE TOTAL 5C: {total_score:.2f} / 100")
        
        # Interpretación del score total
        if total_score >= 70:
            print("🌟 EXCELENTE PERFIL CREDITICIO - Riesgo bajo")
        elif total_score >= 50:
            print("💼 PERFIL MODERADO - Riesgo medio")
        else:
            print("🚨 PERFIL DE ALTO RIESGO - Se recomienda análisis detallado")
    
    def ejecutar_analisis_completo(self):
        """Ejecutar análisis completo con pesos automáticos"""
        # Cargar y preparar datos
        self.cargar_y_preparar_datos()
        
        # Definir categorías 5C
        self.definir_categorias_5c()
        
        # Calcular importancia de variables
        mi_df = self.calcular_importancia_variables()
        
        # Calcular pesos automáticos
        pesos_combinados, pesos_mi, pesos_rf, correlaciones = self.calcular_pesos_automaticos(mi_df)
        
        # Calcular scores por categoría
        resultados_5c = self.calcular_score_por_categoria(mi_df)
        
        # Calcular riesgo por valor
        riesgo_valor_df = self.calcular_riesgo_por_valor()
        
        # Generar reporte
        self.generar_reporte_5c(resultados_5c, {
            'MI': pesos_mi,
            'RF': pesos_rf,
            'Correlacion': correlaciones,
            'Combinado': pesos_combinados
        })
        
        # Guardar resultados
        self.guardar_resultados(mi_df, resultados_5c, riesgo_valor_df, pesos_combinados)
        
        return resultados_5c, mi_df, riesgo_valor_df, pesos_combinados
    
    def guardar_resultados(self, mi_df, resultados_5c, riesgo_valor_df, pesos_automaticos):
        """Guardar todos los resultados en archivos"""
        # Guardar importancia de variables
        mi_df.to_csv("./salidas_csv/pesos_variables_automatico.csv", index=False)
        
        # Guardar scores por categoría 5C
        scores_5c_df = pd.DataFrame([
            {
                'Categoria': cat,
                'Score_Raw': info['score_raw'],
                'Score_Ponderado': info['score_ponderado'],
                'Peso_Automatico': info['peso_automatico'],
                'Num_Variables': info['num_variables'],
                'Importancia_Promedio': info['importancia_promedio'],
                'Interpretacion': info['interpretacion']
            }
            for cat, info in resultados_5c.items()
        ])
        scores_5c_df.to_csv("./salidas_csv/scores_5c_automatico.csv", index=False)
        
        # Guardar pesos automáticos
        pesos_df = pd.DataFrame([
            {'Categoria': cat, 'Peso': peso}
            for cat, peso in pesos_automaticos.items()
        ])
        pesos_df.to_csv("./salidas_csv/pesos_automaticos_5c.csv", index=False)
        
        # Guardar riesgo por valor si existe
        if not riesgo_valor_df.empty:
            riesgo_valor_df.to_csv("./salidas_csv/pesos_valores_automatico.csv", index=False)
        
        print(f"\n💾 Resultados guardados:")
        print(f"   - pesos_variables_automatico.csv")
        print(f"   - scores_5c_automatico.csv")
        print(f"   - pesos_automaticos_5c.csv")
        if not riesgo_valor_df.empty:
            print(f"   - pesos_valores_automatico.csv")

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    # Inicializar y ejecutar análisis
    analizador = Analisis5CAutomatico("./../Registros_sin_nulos.csv")
    resultados, mi_df, riesgo_df, pesos_auto = analizador.ejecutar_analisis_completo()

📊 Cargando datos...
✅ Datos cargados: 14148 registros, 30 variables
   Carácter: 4 variables
   Capacidad: 1 variables
   Capital: 2 variables
   Condiciones: 2 variables
   Colateral: 3 variables
🔍 Calculando importancia de variables...
⚖️ Calculando pesos automáticos para las 5C...
🌲 Calculando importancia con Random Forest...
📊 Calculando correlaciones...

📈 PESOS AUTOMÁTICOS CALCULADOS:
   Carácter: 0.170 (17.0%)
   Capacidad: 0.166 (16.6%)
   Capital: 0.219 (21.9%)
   Condiciones: 0.044 (4.4%)
   Colateral: 0.400 (40.0%)
📈 Calculando scores por categoría 5C...
🎯 Calculando riesgo por valor...

📊 REPORTE COMPLETO 5C DEL CRÉDITO (PESOS AUTOMÁTICOS)

🔧 METODOLOGÍA DE PESOS:
   - Mutual Information: Basado en dependencia estadística
   - Random Forest: Basado en importancia en clasificación
   - Correlación: Basado en relación lineal con morosidad

🔹 CARÁCTER
   Score: 15.25 / 100
   Score ponderado: 2.60
   Peso automático: 17.0%
   Variables analizadas: 4
   Interpretación: Historia